In [3]:
# preliminary function for reading books

# Loading the bookwide csv file from the data cleaning R file

def book_read(book_name):
    
    book = pd.read_csv('annotated_books/'+book_name+'annotated.csv')

    # make addressees lists

    for i in book.index:

        if book.at[i,'dialogue'] != 'NO QUOTE':

            list_ad = []

            addressee = book.at[i,'addressee']

            for j in range(len(addressee)):

                if addressee[j] == '\'':

                    word = ''

                    for k in range(j+1,len(addressee)):

                        if (addressee[k] != '\''):

                            word = word + addressee[k]

                        else:
                
                            break
                
        
                    list_ad.append(word)


            while ', ' in list_ad:
                    
                list_ad.remove(', ')

            while ']' in list_ad:
                    
                list_ad.remove(']')
                

            book.at[i,'addressee'] = list_ad


    character_info = pd.read_csv('pdnc_dataset/data/'+book_name+'/character_info.csv')
    character_info
        
    Characters = character_info['Main Name']
    Characters = list(Characters)
        
    Aliases = []

    Characters_genders = list(character_info['Gender'])
        
    for i in character_info.index:
        new_list = []
        for j in range(len(character_info.at[i,'Aliases'])):
            string = character_info.at[i,'Aliases']
            if string[j] == '\'':
                new = ''
                index = j+1
                while (string[index] != '\'' and index < (len(string)-1)):
                    new = new + string[index]
                    index += 1
                new_list.append(new)
        
        # removing ', ' and '' terms
        
        for i in new_list:
            if (i == ', ' or i == ''):
                new_list.remove(i)
            
        Aliases.append(new_list)

    return (book, Characters, Aliases, Characters_genders)


In [5]:
import numpy as np
import pandas as pd

from model_function_ss_speaker import model_fitting_sp # importing speaker attribution file
from model_function_ss_addressee import model_fitting_ad # importing addressee attribution file


import csv

# completely fitting
    
performances = [['book_name','speaker_ac','prop_explicit','accuracy_ac','full_ac']]

Novel_names = []

with open('pdnc_dataset/PDNC-Novel-index.csv','r') as file:

    reader = csv.reader(file)

    for i in reader:

        novel_name = ""

        for j in i[1]:
            if ((j != " ") and (j != "'") and (j != "-")):
                novel_name = novel_name + j

        
        Novel_names.append(novel_name)
        
    Novel_names.remove('NovelTitle')


# fitting model for each book name

for book_name in Novel_names:

    book_sp = model_fitting_sp(book_name) # running the speaker attribution

    # measuring accuracy

    new_book, Characters, Aliases, Characters_genders = book_read(book_name)
    
    real_speakers = list(new_book[new_book['dialogue'] != 'NO QUOTE']['speaker'])
    
    count = 0
    
    for i in range(len(real_speakers)):
        if list(book_sp[book_sp['dialogue']!= 'NO QUOTE']['speaker'])[i] == real_speakers[i]:
            count += 1
    
    accuracy_sp = count/(len(real_speakers))

    prop_explicit = len(book_sp[book_sp['syntactic'] == 'Explicit'])/len(real_speakers)


    # doing some transformations on book_sp prior to addressee attribution, because it can't deal with multi-addressee rows, or list addressees
    
    new_database = []

    for i in book_sp.index:

        if type(book_sp.at[i,'addressee']) == list:

            if len(book_sp.at[i,'addressee']) == 1:

                new_row = book_sp.loc[i]

                new_row.at['addressee'] = new_row.at['addressee'][0] # making single element a list

                new_database.append(new_row)
                
            else: 

                new_row = book_sp.loc[i]

                new_row.at['addressee'] = np.nan # not trying to predict this row

                new_database.append(new_row)

        else:

            new_database.append(book_sp.loc[i])

    

    new_database = pd.DataFrame(new_database)
    
    
    book = model_fitting_ad(new_database,book_name)

    # checking accuracy here, only looking at single-addressee entries of the annotated book

    # looking at only predicting single addressees using ML methods

    correct_database = []

    for i in new_book.index:

        if type(new_book.at[i,'addressee']) == list:

            if len(new_book.at[i,'addressee']) == 1:

                new_row = new_book.loc[i]

                new_row.at['addressee'] = new_row.at['addressee'][0] # making single element a list

                correct_database.append(new_row)
                
            else:

                new_row = new_book.loc[i]

                new_row.at['addressee'] = np.nan # not trying to predict this row

                correct_database.append(new_row)

        else:
    
            correct_database.append(new_book.loc[i])
    
    correct_database = pd.DataFrame(correct_database)
    
    # measuring accuracy
    
    count_sa = 0
    count_sa_true = 0
    
    count_full = 0

    for i in correct_database.index:

        if (correct_database.at[i,'addressee'] is np.nan) == False:

            # not a multi addressee

            if book.at[i,'addressee'] == correct_database.at[i,'addressee']:

                count_sa_true += 1

                count_sa += 1

            else:

                count_sa += 1


            # speaker true

            if book.at[i,'speaker'] == correct_database.at[i,'speaker']:

                if book.at[i,'addressee'] == correct_database.at[i,'addressee']:

                    count_full += 1



    accuracy_ml_sa = count_sa_true / count_sa

    accuracy_ml_full = count_full / count_sa

    performances.append([book_name,accuracy_sp,prop_explicit,accuracy_ml_sa,accuracy_ml_full])
    
# writing to a csv file

with open('results_ML.csv','w') as file:
    writer = csv.writer(file)
    writer.writerows(performances)

/var/folders/sq/hskpm88j701ghk9f50_gb25r0000gn/T/ipykernel_41052/3862485964.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.at['addressee'] = np.nan # not trying to predict this row
/var/folders/sq/hskpm88j701ghk9f50_gb25r0000gn/T/ipykernel_41052/3862485964.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.at['addressee'] = new_row.at['addressee'][0] # making single element a list
/var/folders/sq/hskpm88j701ghk9f50_gb25r0000gn/T/ipykernel_41052/3862485964.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http